# Final LCA Xero API

The issue is with the structure of http when getting user authorised. A bit different with the example from the web.


You need to get client id from developer side as well as client secret. It is currently stored below but you can always generate another one from the web, don't delete previous ones and it will still work

# User Setup

In [2]:
import base64

client_id = 'B628C81B8827482AAC0E3E7BA73676F2'     # LCA REZA API
client_secret = 'fUr9qANQNTne6sXMaCoXzGzHty3i1dE2sVJe8iWFAUDUpehe'    # Reza API


redirect_url = 'https://developer.xero.com'
scope = 'offline_access accounting.reports.read'
setting = '%20accounting.reports.read%20accounting.settings.read'

b64_id_secret = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')
state = '123'


# Get user authorisation

In [15]:
auth_url = ('https://login.xero.com/identity/connect/authorize?' +
'response_type=code' +
'&client_id=' + client_id +
'&redirect_uri=' + redirect_url + '&scope=offline_access' + setting + '&state=123' )

print (auth_url)

https://login.xero.com/identity/connect/authorize?response_type=code&client_id=B628C81B8827482AAC0E3E7BA73676F2&redirect_uri=https://developer.xero.com&scope=offline_access%20accounting.reports.read%20accounting.settings.read&state=123


# Exchange token the first time

In [17]:
code = 'b0a06dcfd403215004cb3be723cac4d7f1529145d1a8ccefe872fb677d3a8aef' 
    
# 3. Exchange the code
exchange_code_url = 'https://identity.xero.com/connect/token'
response = requests.post(exchange_code_url, 
                        headers = {
                            'Authorization': 'Basic ' + b64_id_secret
                        },
                        data={
              'grant_type': 'authorization_code',
              'code': code,
              'redirect_uri': redirect_url,
              'state' : '123'
})

# save refresh token
f = open('LCA_OATH2.txt','w')
refresh_token = response.json()['refresh_token']
f.write(refresh_token)

print (response)

<Response [400]>


# Code for Refresh Token (Token Already Saved)


In [3]:
import requests
import base64
import os

b64_id_secret = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')


# open last refresh token
f = open('LCA_OATHFINAL.txt','r')
refresh_token = f.read()


# get new access token and refresh token
url = 'https://identity.xero.com/connect/token'
response = requests.post(url,
           headers={
               'Authorization': 'Basic ' + b64_id_secret
           },
           data={
               'grant_type': 'refresh_token',
               'client_id' : client_id,
               'refresh_token': refresh_token
           })


response.json()
response_dict = response.json()
access_token = response_dict['access_token']
refresh_token = response_dict['refresh_token']


# save new refresh token
f = open('LCA_OATHFINAL.txt','w')
f = f.write(refresh_token)

# Test Connection

In [5]:
import json

url = 'https://api.xero.com/connections'
response = requests.get(url,
           headers={
               'Authorization' : 'Bearer '+access_token,
               'Accept' : 'application/json'})
json.loads(response.text)

[{'id': '9f55e0fe-3d19-457a-b88b-551c3a97964d',
  'authEventId': '776d54f2-53b9-4517-869b-936c116992c6',
  'tenantId': '00107543-75d6-4985-9a8c-18e306b29e18',
  'tenantType': 'ORGANISATION',
  'tenantName': 'LCA Elizabeth Pty Limited',
  'createdDateUtc': '2020-07-09T00:22:27.6081490',
  'updatedDateUtc': '2020-07-09T00:22:27.6111890'},
 {'id': '9a4ad96d-96a6-4787-a19d-20cf107fb771',
  'authEventId': '55d1e8e1-da9a-45f7-b7d3-73aeb32661c7',
  'tenantId': '008b63ff-fd52-4922-8ff1-035e595f77c4',
  'tenantType': 'ORGANISATION',
  'tenantName': 'LCA Runaway Bay Pty Limited',
  'createdDateUtc': '2020-07-08T23:54:25.6421790',
  'updatedDateUtc': '2020-07-08T23:54:25.6430600'},
 {'id': '39df8fc5-9e6c-4400-90a7-bd8b40334ee3',
  'authEventId': 'f0cbcbe4-800f-4bbc-98cb-b20434ffc5ab',
  'tenantId': '00950204-b0b0-4abd-a051-3065ddd2f3e2',
  'tenantType': 'ORGANISATION',
  'tenantName': 'LCA Mackay Pty Ltd',
  'createdDateUtc': '2020-07-09T00:08:30.5423660',
  'updatedDateUtc': '2020-07-09T00:08:30

# Create CSV/DataFrame


In [12]:
final = pd.DataFrame(columns=['Date','XeroOrganisation','SectionName','AccountCode','AccountName','Debit','Credit','YTD Debit','YTD Credit'])

############ get list of tenants ################
url = 'https://api.xero.com/connections'
response = requests.get(url,
           headers={
               'Authorization' : 'Bearer '+access_token,
               'Accept' : 'application/json'})
myjson = json.loads(response.text)

mytenants = []
for i in myjson:
    mytenants.append(i['tenantId'])
###################################################


############ put parameter for month ##############
########## Change URL for TRIAL BALANCE ###########

mymonth = '2022-04-30'
url = 'https://api.xero.com/api.xro/2.0/Reports/TrialBalance?date='+mymonth+'&paymentsOnly=false'

###################################################


######### Get JSON API for each tenant ############
for mytenant in mytenants:
    response = requests.get(url,
            headers={
           'Authorization' : 'Bearer '+access_token,
           'Accept' : 'application/json',
           'Xero-tenant-id' : mytenant})
    data = json.loads(response.text)
###################################################



#### Modified the Json to DataFrame and CSV #######
    for i in data['Reports']:
        for j in i['ReportTitles']:
            if 'Trial' not in j and 'As at' not in j:
                clinicname = j

    # get Values from JSON API
    mycell = []
    for i in data['Reports']:
        for j, l1 in enumerate(i['Rows']):
            # only get the data from second row onwards, ignore headers 
            if j != 0:

                # grab section
                mysection = l1['Title']

                # go to dictionary rows section and loop it to find cells
                # from cells you can get all the values for each row
                for l2 in l1['Rows']:
                    #print (l2,'\n')
                    myrow = []
                    myrow.append(mysection)
                    for l3 in l2['Cells']:
                      #print (l3['Value'])
                      myrow.append(l3['Value'])
                    mycell.append(myrow)

    # create draft dataframe                
    df = pd.DataFrame(mycell)
    df.columns = ['Section','Desc','Dr','Cr','YTD Dr','YTD Cr']

    # add additional information to dataframe
    df['ClinicName'] = clinicname
    df['Month'] = mymonth
    df['AccountCode'] = df['Desc'].apply(lambda x:x[-4:-1])
    df['Desc'] = df['Desc'].apply(lambda x:x[:-6])

    # final dataframe
    df = df[['Month','ClinicName','Section','AccountCode','Desc','Dr','Cr','YTD Dr','YTD Cr']]
    df.columns = ['Date','XeroOrganisation','SectionName','AccountCode','AccountName','Debit','Credit','YTD Debit','YTD Credit']

    df.to_csv(clinicname+'.csv',index=False)
    final = final.append(df)
######################################################   

final